In [2]:
#Import initial necessary packages
import os, sys, email
import numpy as np 
import pandas as pd
# Plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns; sns.set_style('whitegrid')
#import plotly
#plotly.offline.init_notebook_mode()
#import plotly.graph_objs as go
#import wordcloud 

#comment to test if I can edit this file

# NLP
from nltk.tokenize.regexp import RegexpTokenizer


In [2]:
#import raw data file
emails_df = pd.read_csv("E:/Use Case Testing/NLP/Raw Data/emails.csv")

emails_df.shape
emails_df.head()

file                                            message
0     allen-p/_sent_mail/1.  Message-ID: <18782981.1075855378110.JavaMail.e...
1    allen-p/_sent_mail/10.  Message-ID: <15464986.1075855378456.JavaMail.e...
2   allen-p/_sent_mail/100.  Message-ID: <24216240.1075855687451.JavaMail.e...
3  allen-p/_sent_mail/1000.  Message-ID: <13505866.1075863688222.JavaMail.e...
4  allen-p/_sent_mail/1001.  Message-ID: <30922949.1075863688243.JavaMail.e...

In [3]:
#Each email looks like this
print(emails_df['message'][0])

Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>
Date: Mon, 14 May 2001 16:39:00 -0700 (PDT)
From: phillip.allen@enron.com
To: tim.belden@enron.com
Subject: 
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Tim Belden <Tim Belden/Enron@EnronXGate>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst

Here is our forecast

 


In [4]:
## Helper functions
def get_text_from_email(msg):
    '''To get the content from email objects'''
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            parts.append( part.get_payload() )
    return ''.join(parts)

def split_email_addresses(line):
    '''To separate multiple email addresses'''
    if line:
        addrs = line.split(',')
        addrs = frozenset(map(lambda x: x.strip(), addrs))
    else:
        addrs = None
    return addrs

In [5]:
# Parse the emails into a list email objects
messages = list(map(email.message_from_string, emails_df['message']))
emails_df.drop('message', axis=1, inplace=True)
# Get fields from parsed email objects
keys = messages[0].keys()
for key in keys:
    emails_df[key] = [doc[key] for doc in messages]
# Parse content from emails
emails_df['content'] = list(map(get_text_from_email, messages))
# Split multiple email addresses
emails_df['From'] = emails_df['From'].map(split_email_addresses)
emails_df['To'] = emails_df['To'].map(split_email_addresses)

# Extract the root of 'file' as 'user'
emails_df['user'] = emails_df['file'].map(lambda x:x.split('/')[0])
del messages

emails_df.head()

file                                     Message-ID  \
0     allen-p/_sent_mail/1.  <18782981.1075855378110.JavaMail.evans@thyme>   
1    allen-p/_sent_mail/10.  <15464986.1075855378456.JavaMail.evans@thyme>   
2   allen-p/_sent_mail/100.  <24216240.1075855687451.JavaMail.evans@thyme>   
3  allen-p/_sent_mail/1000.  <13505866.1075863688222.JavaMail.evans@thyme>   
4  allen-p/_sent_mail/1001.  <30922949.1075863688243.JavaMail.evans@thyme>   

                                    Date                       From  \
0  Mon, 14 May 2001 16:39:00 -0700 (PDT)  (phillip.allen@enron.com)   
1   Fri, 4 May 2001 13:51:00 -0700 (PDT)  (phillip.allen@enron.com)   
2  Wed, 18 Oct 2000 03:00:00 -0700 (PDT)  (phillip.allen@enron.com)   
3  Mon, 23 Oct 2000 06:13:00 -0700 (PDT)  (phillip.allen@enron.com)   
4  Thu, 31 Aug 2000 05:07:00 -0700 (PDT)  (phillip.allen@enron.com)   

                          To    Subject Mime-Version  \
0     (tim.belden@enron.com)                     1.0   
1  (john.lavorato@enron.com)        Re:          1.0   
2   (leah.arsdall@enron.com)   Re: test          1.0   
3    (randall.gay@enron.com)                     1.0   
4     (greg.piper@enron.com)  Re: Hello          1.0   

                   Content-Type Content-Transfer-Encoding           X-From  \
0  text/plain; charset=us-ascii                      7bit  Phillip K Allen   
1  text/plain; charset=us-ascii                      7bit  Phillip K Allen   
2  text/plain; charset=us-ascii                      7bit  Phillip K Allen   
3  text/plain; charset=us-ascii                      7bit  Phillip K Allen   
4  text/plain; charset=us-ascii                      7bit  Phillip K Allen   

                                                X-To X-cc X-bcc  \
0           Tim Belden <Tim Belden/Enron@EnronXGate>              
1  John J Lavorato <John J Lavorato/ENRON@enronXg...              
2                                   Leah Van Arsdall              
3                                      Randall L Gay              
4                                         Greg Piper              

                                            X-Folder X-Origin  \
0  \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...  Allen-P   
1  \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...  Allen-P   
2    \Phillip_Allen_Dec2000\Notes Folders\'sent mail  Allen-P   
3    \Phillip_Allen_Dec2000\Notes Folders\'sent mail  Allen-P   
4    \Phillip_Allen_Dec2000\Notes Folders\'sent mail  Allen-P   

                    X-FileName  \
0  pallen (Non-Privileged).pst   
1  pallen (Non-Privileged).pst   
2                   pallen.nsf   
3                   pallen.nsf   
4                   pallen.nsf   

                                             content     user  
0                          Here is our forecast\n\n   allen-p  
1  Traveling to have a business meeting takes the...  allen-p  
2                     test successful.  way to go!!!  allen-p  
3  Randy,\n\n Can you send me a schedule of the s...  allen-p  
4                Let's shoot for Tuesday at 11:45.    allen-p

In [6]:
#Set index and drop useless columns
emails_df = emails_df.set_index('Message-ID').drop(['file','Mime-Version','Content-Type','Content-Transfer-Encoding'],\
                                                  axis=1)
#Parse datetime
emails_df['Date'] = pd.to_datetime(emails_df['Date'], infer_datetime_format=True)

In [7]:
####################################################################################################
####################################################################################################
## 
## Now that we have the data seperated out into all the necessry columns/features we can begin
## tokenizing.
##
####################################################################################################

In [8]:
#import word_tokenizer which tokenizes words
from nltk.tokenize import word_tokenize



In [9]:
#tokenize entire set of email contents
#tokenized = [word_tokenize(i) for i in emails_df['content']]

In [10]:
#remove stopwords and turn everything to lower case. Create a function to do this
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
stop = set(stopwords.words('english'))
stemmer = PorterStemmer()

def content_to_words( content ):
    #1) Take in content and convert to string
    #2) convert to lower case
    #3) remove stop words
    #4) stemming
    #5) return to original dataframe
    
    message = str(content)
    letters_only = re.sub("[^a-zA-Z\s]", "", message)
    message_low = letters_only.lower().split() #NEED THE SPLIT or else stop words reads each character as 
    # a list and removes half the characters
    words = [w for w in message_low if not w in stop]
    stemmed_words = [stemmer.stem(word) for word in words]
    
    return(" ".join(words)) 



In [5]:
from time import time

In [14]:
#Call content_to_words function for all emails and tokenize it
#initialize new empty array
clean_contents = []
t0 = time()
for i in xrange(0, len(emails_df)):
    temp = content_to_words(emails_df['content'][i])
    clean_contents.append(temp)


tokenized = [word_tokenize(x) for x in clean_contents]
print("Time to tokenize: %0.3fs." %(time() - t0))

Time to tokenize: 1442.650s.


In [15]:
#create new column/feature of tokenized emails into a new email dataframe
emails_df_toke = emails_df
emails_df_toke['content_toke'] = tokenized


In [16]:
emails_df_toke.head()
print(tokenized[10])

['mr', 'buckner', 'delivered', 'gas', 'behind', 'san', 'diego', 'enron', 'energy', 'services', 'appropriate', 'enron', 'entity', 'forwarded', 'request', 'zarin', 'imam', 'ees', 'phone', 'number', 'phillip', 'allen']


In [ ]:
#################################################################################################
#################################################################################################
##
## Time to incorporate LDA
##
##################################################################################################

In [ ]:
#import LDA from sklearn
##from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
##from sklearn.decomposition import NMF, LatentDirichletAllocation

##sklearn implementation of LDA is not real LDA and it took way too long. Time to move to 
##gensim which allows you to do multicore LDA


#Use Tf-idf features for Negative Matrix Factorization model
#tfidf_vectorizer = TfidfVectorizer()
#tfidf = tfidf_vectorizer.fit_transform(clean_contents)
                                       
#t0 = time()
#nmf = NMF.fit(tfidf)
#print("done in %0.3fs." %(time()-t0))
#print("\nTopics in NMF model:")
#tfidf_feature_names = tfidf_vectorizer.get_feature_names()
#tf_feature_names = tf_vectorizer.get_feature_names()
#print_top_words(lda, tf_feature_names, 20)                                       

#tf_vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None)
#t0=time()

#tf = tf_vectorizer.fit_transform(clean_contents)
#print("done in %0.3fs." %(time()-t0))



#print(clean_contents_temp[3])

#lda = LatentDirichletAllocation(n_topics = 200)
#print("test")
#t0 = time()
#lda.fit(tf)
#print("done in %0.3fs." %(time()-t0))

In [17]:
# Function to print out some of the topics
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [ ]:
#Move our tokenized list into a csv so we save it for later use. Also save the cleaned dataframe

In [21]:
import unicodecsv as csv
filepath = "E:/Use Case Testing/NLP/tokenized.csv"

with open(filepath, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(tokenized)

In [22]:
filepath2 = "E:/Use Case Testing/NLP/structured_email.csv"
    
emails_df_toke.to_csv(filepath2)

In [1]:
from gensim import corpora, models

#go through and collect data on each unique word. Also asign a unique ID to each word
dictionary = corpora.Dictionary(tokenized) 

#Code to see unique ID list for tokens
#print(dictionary.token2id)

C:\Anaconda\lib\site-packages\gensim\utils.py:843: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [ ]:
print(len(dictionary))

In [ ]:
print(dictionary[500000])

In [24]:
#convert dictionary to bag of words
t0 = time()
corpus = [dictionary.doc2bow(text) for text in tokenized]
print("Done in %0.3fs" %(time() - t0))
print(corpus[3]) #prints out (termID, term frequency)

Done in 106.809s
[(66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1)]


In [25]:
#get a shorter version of the corpus for testing purposes
test_corpus = corpus[:100000]

In [26]:
#save dictionary
dictionary.save("E:/Use Case Testing/NLP/dictionary.dict")
#save corpus as a matrix market file. MM type is good for sparse matrices such as the corpus
corpora.MmCorpus.serialize("E:/Use Case Testing/NLP/corpus.mm", corpus)

In [3]:
######## Use this code for whenver you need to rerun LDA from a fresh boot

import os, sys, email
import numpy as np 
import pandas as pd
from gensim import corpora, models
from time import time

dictionary = corpora.Dictionary.load("E:/Use Case Testing/NLP/dictionary.dict")
corpus = corpora.MmCorpus("E:/Use Case Testing/NLP/corpus.mm")
test_corpus = corpus[:100000]

In [12]:
t0 = time()
#id2word maps ids to strings using our dictionary that we made earlier. it is required
lda = models.LdaMulticore(corpus, num_topics=50, id2word = dictionary,\
                          workers = 23, batch = True)
print("LDA done in %0.3fm" %((time() - t0)/60))

LDA done in 266.171m


In [13]:
#save lda
lda.save("E:/Use Case Testing/NLP/lda.model")

TypeError: 'int' object has no attribute '__getitem__'